# Convnet Basics

## Example

In [2]:
import tensorflow as tf

In [3]:
# read dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [4]:
# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

In [5]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])), # 5x5 filter (has no affect on size)
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [6]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [7]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [11]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1']) # 28*28*32
    conv1 = maxpool2d(conv1, k=2) # 14*14*32

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]]) # 7*7*64=3136
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [12]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 64024.4062 Validation Accuracy: 0.121094
Epoch  1, Batch   2 -Loss: 48414.3906 Validation Accuracy: 0.113281
Epoch  1, Batch   3 -Loss: 39227.2656 Validation Accuracy: 0.109375
Epoch  1, Batch   4 -Loss: 33691.4453 Validation Accuracy: 0.113281
Epoch  1, Batch   5 -Loss: 32312.1016 Validation Accuracy: 0.109375
Epoch  1, Batch   6 -Loss: 30225.5742 Validation Accuracy: 0.105469
Epoch  1, Batch   7 -Loss: 26505.0273 Validation Accuracy: 0.109375
Epoch  1, Batch   8 -Loss: 29553.8672 Validation Accuracy: 0.113281
Epoch  1, Batch   9 -Loss: 22446.4414 Validation Accuracy: 0.109375
Epoch  1, Batch  10 -Loss: 25087.3770 Validation Accuracy: 0.117188
Epoch  1, Batch  11 -Loss: 26643.2637 Validation Accuracy: 0.125000
Epoch  1, Batch  12 -Loss: 18339.8809 Validation Accuracy: 0.121094
Epoch  1, Batch  13 -Loss: 20397.8516 Validation Accuracy: 0.128906
Epoch  1, Batch  14 -Loss: 19247.9922 Validation Accuracy: 0.140625
Epoch  1, Batch  15 -Loss: 19163.4473 Validation

Epoch  1, Batch 122 -Loss:  5475.3740 Validation Accuracy: 0.511719
Epoch  1, Batch 123 -Loss:  5018.6230 Validation Accuracy: 0.503906
Epoch  1, Batch 124 -Loss:  4376.5215 Validation Accuracy: 0.515625
Epoch  1, Batch 125 -Loss:  3434.0464 Validation Accuracy: 0.515625
Epoch  1, Batch 126 -Loss:  6033.1333 Validation Accuracy: 0.515625
Epoch  1, Batch 127 -Loss:  4787.0166 Validation Accuracy: 0.515625
Epoch  1, Batch 128 -Loss:  4579.0591 Validation Accuracy: 0.523438
Epoch  1, Batch 129 -Loss:  4194.0835 Validation Accuracy: 0.527344
Epoch  1, Batch 130 -Loss:  4821.4248 Validation Accuracy: 0.523438
Epoch  1, Batch 131 -Loss:  5101.4072 Validation Accuracy: 0.523438
Epoch  1, Batch 132 -Loss:  5762.9316 Validation Accuracy: 0.527344
Epoch  1, Batch 133 -Loss:  4733.6221 Validation Accuracy: 0.531250
Epoch  1, Batch 134 -Loss:  4688.1484 Validation Accuracy: 0.542969
Epoch  1, Batch 135 -Loss:  3741.9438 Validation Accuracy: 0.539062
Epoch  1, Batch 136 -Loss:  5676.8882 Validation

Epoch  1, Batch 243 -Loss:  3326.8025 Validation Accuracy: 0.636719
Epoch  1, Batch 244 -Loss:  2535.5879 Validation Accuracy: 0.636719
Epoch  1, Batch 245 -Loss:  2778.3071 Validation Accuracy: 0.632812
Epoch  1, Batch 246 -Loss:  2534.2444 Validation Accuracy: 0.632812
Epoch  1, Batch 247 -Loss:  3025.5254 Validation Accuracy: 0.636719
Epoch  1, Batch 248 -Loss:  2390.0049 Validation Accuracy: 0.640625
Epoch  1, Batch 249 -Loss:  3145.5422 Validation Accuracy: 0.648438
Epoch  1, Batch 250 -Loss:  2210.0361 Validation Accuracy: 0.648438
Epoch  1, Batch 251 -Loss:  3043.2715 Validation Accuracy: 0.656250
Epoch  1, Batch 252 -Loss:  2937.7766 Validation Accuracy: 0.652344
Epoch  1, Batch 253 -Loss:  2730.9595 Validation Accuracy: 0.648438
Epoch  1, Batch 254 -Loss:  2703.9336 Validation Accuracy: 0.636719
Epoch  1, Batch 255 -Loss:  2680.1140 Validation Accuracy: 0.632812
Epoch  1, Batch 256 -Loss:  2567.7104 Validation Accuracy: 0.644531
Epoch  1, Batch 257 -Loss:  2173.9805 Validation

Epoch  1, Batch 364 -Loss:  1794.5378 Validation Accuracy: 0.707031
Epoch  1, Batch 365 -Loss:  1888.5381 Validation Accuracy: 0.710938
Epoch  1, Batch 366 -Loss:  2106.5281 Validation Accuracy: 0.707031
Epoch  1, Batch 367 -Loss:  1443.1901 Validation Accuracy: 0.710938
Epoch  1, Batch 368 -Loss:  1691.7742 Validation Accuracy: 0.714844
Epoch  1, Batch 369 -Loss:  1697.7355 Validation Accuracy: 0.710938
Epoch  1, Batch 370 -Loss:  1321.8646 Validation Accuracy: 0.710938
Epoch  1, Batch 371 -Loss:  1804.9434 Validation Accuracy: 0.710938
Epoch  1, Batch 372 -Loss:  2533.1934 Validation Accuracy: 0.707031
Epoch  1, Batch 373 -Loss:  1949.0264 Validation Accuracy: 0.699219
Epoch  1, Batch 374 -Loss:  1642.0414 Validation Accuracy: 0.707031
Epoch  1, Batch 375 -Loss:  1976.4637 Validation Accuracy: 0.707031
Epoch  1, Batch 376 -Loss:  2123.2776 Validation Accuracy: 0.714844
Epoch  1, Batch 377 -Loss:  2006.4897 Validation Accuracy: 0.710938
Epoch  1, Batch 378 -Loss:  1862.2268 Validation

Epoch  2, Batch  56 -Loss:   975.1794 Validation Accuracy: 0.746094
Epoch  2, Batch  57 -Loss:  1406.0043 Validation Accuracy: 0.742188
Epoch  2, Batch  58 -Loss:  1629.8789 Validation Accuracy: 0.746094
Epoch  2, Batch  59 -Loss:  1431.9027 Validation Accuracy: 0.746094
Epoch  2, Batch  60 -Loss:  1688.9385 Validation Accuracy: 0.746094
Epoch  2, Batch  61 -Loss:  1771.3993 Validation Accuracy: 0.750000
Epoch  2, Batch  62 -Loss:  1857.4612 Validation Accuracy: 0.746094
Epoch  2, Batch  63 -Loss:  1200.3873 Validation Accuracy: 0.753906
Epoch  2, Batch  64 -Loss:  1037.8785 Validation Accuracy: 0.753906
Epoch  2, Batch  65 -Loss:  1663.5396 Validation Accuracy: 0.742188
Epoch  2, Batch  66 -Loss:  1425.5938 Validation Accuracy: 0.742188
Epoch  2, Batch  67 -Loss:  1303.4885 Validation Accuracy: 0.738281
Epoch  2, Batch  68 -Loss:  1436.7905 Validation Accuracy: 0.738281
Epoch  2, Batch  69 -Loss:   821.6785 Validation Accuracy: 0.742188
Epoch  2, Batch  70 -Loss:  1380.6083 Validation

...

Epoch  9, Batch 320 -Loss:   281.8499 Validation Accuracy: 0.761719
Epoch  9, Batch 321 -Loss:   155.1901 Validation Accuracy: 0.757812
Epoch  9, Batch 322 -Loss:   335.9281 Validation Accuracy: 0.765625
Epoch  9, Batch 323 -Loss:   223.2071 Validation Accuracy: 0.757812
Epoch  9, Batch 324 -Loss:   190.1458 Validation Accuracy: 0.769531
Epoch  9, Batch 325 -Loss:   109.5535 Validation Accuracy: 0.757812
Epoch  9, Batch 326 -Loss:   243.8705 Validation Accuracy: 0.753906
Epoch  9, Batch 327 -Loss:   208.6767 Validation Accuracy: 0.753906
Epoch  9, Batch 328 -Loss:   228.9571 Validation Accuracy: 0.753906
Epoch  9, Batch 329 -Loss:   263.0895 Validation Accuracy: 0.753906
Epoch  9, Batch 330 -Loss:   254.3703 Validation Accuracy: 0.761719
Epoch  9, Batch 331 -Loss:   241.4045 Validation Accuracy: 0.765625
Epoch  9, Batch 332 -Loss:   225.1257 Validation Accuracy: 0.773438
Epoch  9, Batch 333 -Loss:   219.4768 Validation Accuracy: 0.769531
Epoch  9, Batch 334 -Loss:   216.5441 Validation

Epoch 10, Batch  12 -Loss:   190.6424 Validation Accuracy: 0.761719
Epoch 10, Batch  13 -Loss:   288.9143 Validation Accuracy: 0.761719
Epoch 10, Batch  14 -Loss:   240.1977 Validation Accuracy: 0.761719
Epoch 10, Batch  15 -Loss:   170.9744 Validation Accuracy: 0.765625
Epoch 10, Batch  16 -Loss:   231.5494 Validation Accuracy: 0.765625
Epoch 10, Batch  17 -Loss:   128.0561 Validation Accuracy: 0.761719
Epoch 10, Batch  18 -Loss:   221.2947 Validation Accuracy: 0.765625
Epoch 10, Batch  19 -Loss:   133.5240 Validation Accuracy: 0.761719
Epoch 10, Batch  20 -Loss:   231.4562 Validation Accuracy: 0.765625
Epoch 10, Batch  21 -Loss:   182.9748 Validation Accuracy: 0.765625
Epoch 10, Batch  22 -Loss:   212.3926 Validation Accuracy: 0.765625
Epoch 10, Batch  23 -Loss:   248.7429 Validation Accuracy: 0.757812
Epoch 10, Batch  24 -Loss:   176.2156 Validation Accuracy: 0.757812
Epoch 10, Batch  25 -Loss:   213.9718 Validation Accuracy: 0.753906
Epoch 10, Batch  26 -Loss:   228.3509 Validation

Epoch 10, Batch 133 -Loss:   207.7335 Validation Accuracy: 0.765625
Epoch 10, Batch 134 -Loss:   194.0868 Validation Accuracy: 0.773438
Epoch 10, Batch 135 -Loss:   235.7140 Validation Accuracy: 0.777344
Epoch 10, Batch 136 -Loss:   185.8142 Validation Accuracy: 0.777344
Epoch 10, Batch 137 -Loss:   175.6333 Validation Accuracy: 0.769531
Epoch 10, Batch 138 -Loss:   243.1996 Validation Accuracy: 0.777344
Epoch 10, Batch 139 -Loss:   193.9283 Validation Accuracy: 0.777344
Epoch 10, Batch 140 -Loss:   268.6905 Validation Accuracy: 0.761719
Epoch 10, Batch 141 -Loss:   235.9177 Validation Accuracy: 0.761719
Epoch 10, Batch 142 -Loss:   182.5284 Validation Accuracy: 0.769531
Epoch 10, Batch 143 -Loss:   267.4056 Validation Accuracy: 0.773438
Epoch 10, Batch 144 -Loss:   283.2917 Validation Accuracy: 0.769531
Epoch 10, Batch 145 -Loss:   163.0230 Validation Accuracy: 0.777344
Epoch 10, Batch 146 -Loss:   294.0647 Validation Accuracy: 0.777344
Epoch 10, Batch 147 -Loss:   169.7969 Validation

Epoch 10, Batch 254 -Loss:   197.0574 Validation Accuracy: 0.777344
Epoch 10, Batch 255 -Loss:   255.7369 Validation Accuracy: 0.773438
Epoch 10, Batch 256 -Loss:   222.5923 Validation Accuracy: 0.769531
Epoch 10, Batch 257 -Loss:   179.7095 Validation Accuracy: 0.773438
Epoch 10, Batch 258 -Loss:   224.5944 Validation Accuracy: 0.761719
Epoch 10, Batch 259 -Loss:   191.5050 Validation Accuracy: 0.753906
Epoch 10, Batch 260 -Loss:   239.3499 Validation Accuracy: 0.765625
Epoch 10, Batch 261 -Loss:   230.3399 Validation Accuracy: 0.769531
Epoch 10, Batch 262 -Loss:   170.8612 Validation Accuracy: 0.769531
Epoch 10, Batch 263 -Loss:   239.7099 Validation Accuracy: 0.773438
Epoch 10, Batch 264 -Loss:   196.9456 Validation Accuracy: 0.773438
Epoch 10, Batch 265 -Loss:   284.8494 Validation Accuracy: 0.773438
Epoch 10, Batch 266 -Loss:   240.2942 Validation Accuracy: 0.761719
Epoch 10, Batch 267 -Loss:   173.2098 Validation Accuracy: 0.777344
Epoch 10, Batch 268 -Loss:   192.3651 Validation

Epoch 10, Batch 375 -Loss:   200.2067 Validation Accuracy: 0.769531
Epoch 10, Batch 376 -Loss:   164.6383 Validation Accuracy: 0.769531
Epoch 10, Batch 377 -Loss:   304.2822 Validation Accuracy: 0.761719
Epoch 10, Batch 378 -Loss:   197.3760 Validation Accuracy: 0.765625
Epoch 10, Batch 379 -Loss:   163.6558 Validation Accuracy: 0.757812
Epoch 10, Batch 380 -Loss:   260.8947 Validation Accuracy: 0.761719
Epoch 10, Batch 381 -Loss:   191.0684 Validation Accuracy: 0.769531
Epoch 10, Batch 382 -Loss:   161.8367 Validation Accuracy: 0.765625
Epoch 10, Batch 383 -Loss:   264.2696 Validation Accuracy: 0.769531
Epoch 10, Batch 384 -Loss:   192.9706 Validation Accuracy: 0.761719
Epoch 10, Batch 385 -Loss:   237.7599 Validation Accuracy: 0.765625
Epoch 10, Batch 386 -Loss:   193.4874 Validation Accuracy: 0.765625
Epoch 10, Batch 387 -Loss:   188.3873 Validation Accuracy: 0.769531
Epoch 10, Batch 388 -Loss:   177.6348 Validation Accuracy: 0.769531
Epoch 10, Batch 389 -Loss:   157.2644 Validation